In [18]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import array
from numpy import hstack
import re
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
import pickle


In [2]:
## Load the data

In [3]:
train_sales = pd.read_csv('./data/sales_train_validation.csv')
sell_prices = pd.read_csv('./data/sell_prices.csv')
calendar = pd.read_csv('./data/calendar.csv')
submission_file = pd.read_csv('./data/sample_submission.csv')

In [4]:
#train_sales = train_sales.set_index('id')
train_sales.tail(31)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
30459,FOODS_3_797_WI_3_validation,FOODS_3_797,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,2,0,1,0,1,1,0,2,1
30460,FOODS_3_798_WI_3_validation,FOODS_3_798,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,1,0,0,0,3,1,2
30461,FOODS_3_799_WI_3_validation,FOODS_3_799,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30462,FOODS_3_800_WI_3_validation,FOODS_3_800,FOODS_3,FOODS,WI_3,WI,36,39,34,42,...,7,10,14,3,10,6,9,8,6,10
30463,FOODS_3_801_WI_3_validation,FOODS_3_801,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,6,3,1,1,1,5,1,3,7,2
30464,FOODS_3_802_WI_3_validation,FOODS_3_802,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,3,1,1,0,0,0,0,0,1,0
30465,FOODS_3_803_WI_3_validation,FOODS_3_803,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30466,FOODS_3_804_WI_3_validation,FOODS_3_804,FOODS_3,FOODS,WI_3,WI,14,7,6,9,...,3,12,3,1,6,14,6,0,4,2
30467,FOODS_3_805_WI_3_validation,FOODS_3_805,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,3,0,4,4,1,3,0,0,5,1
30468,FOODS_3_806_WI_3_validation,FOODS_3_806,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,2,3,1,1,3,0,0,0,0,1


In [5]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [6]:
numberOfDuplicateDays = (calendar['d'].value_counts() > 1).value_counts() - len(calendar)
if(numberOfDuplicateDays[0] != 0):
    print('There are ' + str(numberOfDuplicateDays[0]) + ' duplicate day identifiers!' )

In [7]:
calendar.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0


In [8]:
eventsArray = (calendar['event_name_1'].append(calendar['event_name_2'])).unique()
eventTypesArray = (calendar['event_type_1'].append(calendar['event_type_2'])).unique()
weekDaysArray = calendar['weekday'].unique()
states = train_sales['state_id'].unique()

dayIndexString = 'd_'



In [9]:
def date_to_nth_day(date, format='%Y%m%d'):
    date = pd.to_datetime(date, format=format)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1

def eventsToDaysBefore(day_id, nmbr_days_in_future = 30):
    events = np.full(eventsArray.size, nmbr_days_in_future)
    events_types = np.full(eventTypesArray.size, nmbr_days_in_future)
    for i in range(0, nmbr_days_in_future):
        ## get day in future
        idx = calendar.index[calendar['d'] == dayIndexString + str(day_id + i)]
        dayDetails = calendar.loc[idx]
        event_1 = dayDetails['event_name_1'].iloc[0]
        # Process first event
        if(not pd.isnull(event_1)):
            index = np.where(eventsArray==event_1)
            events[index] = i
        event_2 = dayDetails['event_name_2'].iloc[0]

        # Process second event
        if(not pd.isnull(event_2)):
            index = np.where(eventsArray==event_2)
            events[index] = i

        # Process first event type
        event_t_1 = dayDetails['event_type_1'].iloc[0]
        if(not pd.isnull(event_t_1)):
            index = np.where(eventTypesArray==event_t_1)
            events_types[index] = i

        # Process second event type
        event_t_2 = dayDetails['event_type_2'].iloc[0]
        if(not pd.isnull(dayDetails['event_type_2'].iloc[0])):
            index = np.where(eventTypesArray==event_t_2)
            events_types[index] = i
    return events, events_types
    
def getCalendarInfo(day_id):
    idx = calendar.index[calendar['d'] == day_id]
    dayDetails = calendar.loc[idx]
    dayDetails['date'] =  date_to_nth_day(dayDetails['date'].iloc[0], '%Y-%m-%d')
    # Remove weekday because table included wday value
    dayDetails.drop(columns=['weekday'], inplace = True)
    # Remove week id because I don't know what's the added value because we have year and number of day in year
    dayDetails.drop(columns=['wm_yr_wk'], inplace = True)
    
    dayDetails.drop(columns=['event_name_1'], inplace = True)
    dayDetails.drop(columns=['event_name_2'], inplace = True)
    dayDetails.drop(columns=['event_type_1'], inplace = True)
    dayDetails.drop(columns=['event_type_2'], inplace = True)
    
    # Reset year of 2011 to index year 0
    dayDetails['year'] = dayDetails['year'] - 2011  
    dayDetails['d'] = int(dayDetails['d'].str[2:])
    return dayDetails
    


In [10]:
def getXDays(train_sales, skip ,numberOfDays = 28):
    train_x_data = None
    train_y_data = None

    for i_d in range(numberOfDays):
        dayNumber = skip + 1 + i_d
        # skip the days, then take the next day, then skip the processed days
        day_id = dayIndexString + str(dayNumber)
        # Get the sales on the day
        salesOfDay = train_sales[day_id]
        calendarInfo = getCalendarInfo(day_id)
        eventInfo = eventsToDaysBefore(dayNumber)
        calendarInfo = np.concatenate((calendarInfo.values[0], eventInfo[0]))
        calendarInfo = np.concatenate((calendarInfo, eventInfo[1]))
        if train_x_data is None:
            train_x_data = np.array(calendarInfo)
            train_y_data = np.array(salesOfDay)
        else:
            train_x_data = np.vstack((train_x_data, calendarInfo))
            train_y_data = np.vstack((train_y_data, salesOfDay))


    return (train_x_data), train_y_data.transpose()
trainData = getXDays(train_sales, 0)
print(np.shape(trainData[0]))
print(np.shape(trainData[1]))

print(trainData)

(28, 44)
(30490, 28)
(array([[29,  1,  1, ..., 16, 23, 30],
       [30,  2,  1, ..., 15, 22, 30],
       [31,  3,  1, ..., 14, 21, 30],
       ...,
       [54,  5,  2, ..., 22, 30, 25],
       [55,  6,  2, ..., 21, 30, 24],
       [56,  7,  2, ..., 20, 30, 23]], dtype=int64), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 6, 0, ..., 3, 4, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64))


In [24]:
def trainBatch(day_id = 1, day_range = 28):
    train_container_panda = pd.DataFrame()

    for i in range(0, day_range):
            # append sales of day
            sales = train_sales[dayIndexString + str(day_id + i)]
            train_container_panda['sale_' + str(i)] = sales
    for i in range(0, 1):
            # Get basic calendar info
            calendarInfo = getCalendarInfo('d_' + str(day_id + day_range + i))
            # Get one-hot occur event on this day od
            eventInfo = eventsToDaysBefore(day_id + day_range + i)
            # Concatenate calendar info and event info
            calendarInfo = np.concatenate((calendarInfo.values[0], eventInfo[0]))
            # Concatenate info with event type info
            calendarInfo = np.concatenate((calendarInfo, eventInfo[1]))
            # append the event based information 
            columnIndex = 0
            for col in calendarInfo:
                train_container_panda['cal_' + str(day_id + day_range + i) + '_' + str(columnIndex)] = col
                columnIndex = columnIndex + 1
    return train_container_panda

def testBatch(day_id = 1, day_range = 28):
    train_container_panda = pd.DataFrame()

    for i in range(0, day_range):
            # append sales of d|ay
            sales = train_sales[dayIndexString + str(day_id + i)]
            train_container_panda['sale_' + str(day_id + i)] = sales
               
    return train_container_panda


In [25]:
def getLastDayOfTrainset():
    line2 = train_sales.columns[-1]
    temp1 = re.findall(r'\d+', line2) # through regular expression
    return list(map(int, temp1))[0]


In [26]:
def trainClassifier(trainBatchSize, testBatchSize):
    lastTrainDay = getLastDayOfTrainset()
    
    train_array = []
    test_array = []
    
    startIndex = 1 +  lastTrainDay % (trainBatchSize + testBatchSize);
    print('Start day is:' + str(startIndex))
    while startIndex + testBatchSize  <= lastTrainDay:
        train_batch = trainBatch(startIndex, trainBatchSize).to_numpy()
        train_array.append(train_batch)
        startIndex = startIndex + trainBatchSize
        test_batch = testBatch(startIndex, testBatchSize).to_numpy()
        test_array.append(test_batch)
        startIndex = startIndex + testBatchSize
        print('Samples processed: '+ str(startIndex -1))
    return train_array, test_array


In [27]:
dataset = trainClassifier(56, 28)

Start day is:66
Samples processed: 149
Samples processed: 233
Samples processed: 317
Samples processed: 401
Samples processed: 485
Samples processed: 569
Samples processed: 653
Samples processed: 737
Samples processed: 821
Samples processed: 905
Samples processed: 989
Samples processed: 1073
Samples processed: 1157
Samples processed: 1241
Samples processed: 1325
Samples processed: 1409
Samples processed: 1493
Samples processed: 1577
Samples processed: 1661
Samples processed: 1745
Samples processed: 1829
Samples processed: 1913


In [28]:
print(np.shape(dataset[0]))

(22, 30490, 100)


In [34]:
#rmse array to save all mean rmse values during for loop
rmseArray = np.array([])

test_dataset = dataset[1]

#rmse array to save all mean rmse values during for loop
#for model in models:
estimators = 1
regr = RandomForestRegressor(random_state=0, warm_start = True, n_estimators = estimators)
rmseIntermediateArray = np.array([])
i = 0

for batch in dataset[0]:
        regr.fit(batch, test_dataset[i])
        regr.n_estimators += estimators
        i = i + 1
        rmse = -1
        try:
            result = regr.predict(dataset[0][i+1])
            rmse = sqrt(mean_squared_error(test_dataset[i+1], result))
            rmseIntermediateArray = np.append(rmseIntermediateArray, rmse)
            #print('Batch completed with score' + str(rmse) )
            #print('Batch scores' + str(rmseArray) )

        except IndexError as e: 
            print(str(e))        
            print('Batch completed' + str(i) )
rmseArray = np.append(rmseArray, rmseIntermediateArray[-1])
print('Last rmse: '+ str(rmseIntermediateArray[-1]))
print('Average rmse'  + str(rmseArray))

    

list index out of range
Batch completed21
list index out of range
Batch completed22
Last rmse: 2.1938741690018473
Average rmse[2.19387417]


In [ ]:
print(rmseIntermediateArray)

In [28]:
with open('D:\RU\MLiP\m5 forecasting/randomForest_batches.test', 'wb') as f:
    pickle.dump(regr, f)

In [ ]:
#random forest results
# 2.517 rmse for 1 estimator and 14 days training
# 2.471 rmse for 2 estimator and 14 days training
# 2.455 rmse for 3 estimator and 14 days training
# 2.447 rmse for 4 estimator and 14 days training
# 2. rmse for 8 estimator and 14 days training

# 2.54 rmse for 1 estimator and 28 days training
# 2.494 rmse for 2 estimator and 28 days training
# 2.469 rmse for 3 estimator and 28 days training
# 2.459 rmse for 4 estimator and 28 days training
# 2.449 rmse for 6 estimator and 28 days training
# 2.445 rmse for 8 estimator and 28 days training
# 2.441 rmse for 10 estimator and 28 days training

# 2.56 rmse for 1 estimator and 56 days training
# 2.48 rmse for 2 estimator and 56 days training
# 2.43 rmse for 3 estimator and 56 days training
# 2.39 rmse for 8 estimator and 56 days training
# 2.39 rmse for 8 estimator and 56 days training


# 2.668 rmse for 1 estimator and 70 days training
# 2.557 rmse for 2 estimator and 70 days training
# 2.526 rmse for 3 estimator and 70 days training

#test if weeks are relevant
# 2.690 rmse for 1 estimator and 72 days training
# 2.587 rmse for 2 estimator and 72 days training
# 2.557 rmse for 3 estimator and 72 days training
# 2.537 rmse for 4 estimator and 72 days training

In [30]:
#rmse array to save all mean rmse values during for loop
rmseArray = np.array([])

test_dataset = dataset[1]

#rmse array to save all mean rmse values during for loop
#for model in models:
estimators = 10
r1 = LinearRegression()
r2 = RandomForestRegressor(random_state=0, warm_start = True, n_estimators = estimators)
er = VotingRegressor([('lr', r1), ('rf', r2)])
rmseIntermediateArray = np.array([])
i = 0

for batch in dataset[0]:
        r1.fit(batch, test_dataset[i])
        #r1.n_estimators += estimators
        i = i + 1
        rmse = -1
        try:
            result = r1.predict(dataset[0][i+1])
            rmse = sqrt(mean_squared_error(test_dataset[i+1], result))
            rmseIntermediateArray = np.append(rmseIntermediateArray, rmse)
            #print('Batch completed with score' + str(rmse) )
            #print('Batch scores' + str(rmseArray) )

        except IndexError as e: 
            print(str(e))        
            print('Batch completed' + str(i) )
rmseArray = np.append(rmseArray, rmseIntermediateArray[-1])
print('Last rmse: '+ str(rmseIntermediateArray[-1]))
print('Average rmse'  + str(rmseArray))

list index out of range
Batch completed21
list index out of range
Batch completed22
Last rmse: 2.1862775752473937
Average rmse[2.18627758]


In [31]:
with open('D:\RU\MLiP\m5 forecasting/voting_batches.test', 'wb') as f:
    pickle.dump(r1, f)

In [33]:
np.savetxt('randomforestdataset', dataset[0])

MemoryError: Unable to allocate 512. MiB for an array with shape (22, 30490, 100) and data type int64